In [ ]:
from SmartApi import SmartConnect
from logzero import logger
import json
import http.client
from nsepython import *
import pandas as pd
import time

## Data Collection

In [ ]:
# This script used angle one smart api to colelct stock data any other way of data collection can be done

api_key = "abcd"
username = "abcd"
pwd = "abcd"
totp = ""
refreh_token = ''

jwt_token = ''

feedToken = ''


### Generation of Refresh Token and Access Token

In [ ]:
# smartApi = SmartConnect(api_key)
# try:
#     token = ""
#     totp = pyotp.TOTP(token).now()
# except Exception as e:
#     raise e

# correlation_id = "abcde"
# data = smartApi.generateSession(username, pwd, totp)
# data

In [ ]:
# # #generate referesh token
# conn = http.client.HTTPSConnection(
#     "apiconnect.angelbroking.com"
#     )
# payload = {
#     "refreshToken" : refreh_token
# }
# headers = {
#     'Authorization': jwt_token,
#     'Content-Type': 'application/json',
#     'Accept': 'application/json',
#     'X-UserType': 'USER',
#     'X-SourceID': 'WEB',
#     'X-ClientLocalIP': 'CLIENT_LOCAL_IP',
#     'X-ClientPublicIP': 'CLIENT_PUBLIC_IP',
#     'X-MACAddress': 'MAC_ADDRESS',
#     'X-PrivateKey': api_key
#   }
# conn.request("POST", "/rest/auth/angelbroking/jwt/v1/generateTokens" ,json.dumps(payload),headers)

# res = conn.getresponse()
# data = res.read()
# print(data.decode("utf-8"))

### Requesting Data from the API

In [ ]:
df_pnb = pd.DataFrame(columns = ["Date&Time","Open","High","Low","Close","Volume"])

In [ ]:
def get_data(start_date, end_date):
    start_date = start_date.strftime('%Y-%m-%d %H:%M')
    end_date = end_date.strftime('%Y-%m-%d %H:%M')
    print(start_date,end_date)
    conn = http.client.HTTPSConnection("apiconnect.angelbroking.com")
    payload = {
    "exchange": "NSE",
    "symboltoken": "9808",
    "interval": "ONE_DAY",
    "fromdate": start_date,
    "todate": end_date
    }
    headers = {
    'X-PrivateKey': api_key,
    'Accept': 'application/json',
    'X-SourceID': 'WEB',
    'X-ClientLocalIP': 'CLIENT_LOCAL_IP',
    'X-ClientPublicIP': 'CLIENT_PUBLIC_IP',
    'X-MACAddress': 'MAC_ADDRESS',
    'X-UserType': 'USER',
    'Authorization': jwt_token,
    'Accept': 'application/json',
    'X-SourceID': 'WEB',
    'Content-Type': 'application/json'
    }
    conn.request("POST", "/rest/secure/angelbroking/historical/v1/getCandleData", json.dumps(payload), headers)
    res = conn.getresponse()
    data = json.loads(res.read())
    return data['data']

In [ ]:
base_date = "2012-01-01"

# Loop to fetch data every 12 months from 2012 to 2024
for i in range(13):
    start_date = pd.to_datetime(base_date) + pd.DateOffset(months=(i*12))
    end_date = start_date + pd.DateOffset(months=12)
    data = get_data(start_date, end_date)
    df_temp = pd.DataFrame(data, columns=["Date&Time", "Open", "High", "Low", "Close", "Volume"])
    df_pnb = pd.concat([df_pnb, df_temp], ignore_index=True)
    time.sleep(1)

In [ ]:
df_pnb['Date&Time'] = pd.to_datetime(df_pnb['Date&Time'])
df_pnb['Date&Time'] = df_pnb['Date&Time'].dt.date

In [ ]:
df_pnb.to_csv("../data/PNB.csv", index=False)